# Introduction to Linear Programming with Python - Part 5
## Using PuLP with pandas and binary constraints to solve a scheduling problem

In this example, we'll be solving a scheduling problem. We have 5 pilots and 10 different go's to fill

We want to produce a schedule of pilots from both plants that meets our demand with the lowest cost.

A pilot can be in 2 states:
* Off - not flying
* On - flying

Pilots are either available or not available for each go.

Goal is to fill the schedule with each pilot getting as few flights as possible

In [192]:
import pandas as pd
import pulp
#import math

In [193]:
pilots = pd.read_csv('csv/pilot_availability_v2.csv',index_col=['PILOT'])
pilots

,1,2,3,4,5,6,7,8,9
PILOT,,,,,,,,,
A,0,1,0,1,1,0,1,0,0
B,0,1,0,1,1,0,1,1,1
C,1,0,1,1,1,0,1,1,1
D,0,0,1,1,0,1,0,0,0
E,1,0,1,0,1,0,1,0,0


In [194]:
pilot_quals = pd.read_csv('csv/pilot_qual.csv',index_col=['PILOT'])
pilot_quals

,QUAL
PILOT,
A,2
B,1
C,2
D,1
E,2


We'll also import our demand data

In [195]:
lines = pd.read_csv('csv/go_demand.csv', index_col=['GO','Line'])
lines

Requirement
GO Line             
1  101             2
   102             1
2  201             2
3  301             2
   302             1
4  401             2
5  501             2
   502             1
6  601             2
7  701             2
   702             1
8  801             2
9  901             2
   902             1

Pilot status is modelled as a binary variable. It will have a value of 1 if the pilot is flying and a value of 0 when the pilot is off.

Binary variables are the same as integer variables but constrained to be >= 0 and <=1

Again this has a value for each month for each factory, again given by the index of our DataFrame

In [278]:
for pilot in pilots:
    print(pilot)


1
2
3
4
5
6
7
8
9


In [197]:
pilot_status = pulp.LpVariable.dicts("pilot_status",
                                     ((Line,PILOT) for Line  in lines.index for PILOT in pilots.index ),
                                     cat='Binary')

In [198]:
pilot_status

{((1, 101), 'A'): pilot_status_((1,_101),_'A'),
 ((1, 101), 'B'): pilot_status_((1,_101),_'B'),
 ((1, 101), 'C'): pilot_status_((1,_101),_'C'),
 ((1, 101), 'D'): pilot_status_((1,_101),_'D'),
 ((1, 101), 'E'): pilot_status_((1,_101),_'E'),
 ((1, 102), 'A'): pilot_status_((1,_102),_'A'),
 ((1, 102), 'B'): pilot_status_((1,_102),_'B'),
 ((1, 102), 'C'): pilot_status_((1,_102),_'C'),
 ((1, 102), 'D'): pilot_status_((1,_102),_'D'),
 ((1, 102), 'E'): pilot_status_((1,_102),_'E'),
 ((2, 201), 'A'): pilot_status_((2,_201),_'A'),
 ((2, 201), 'B'): pilot_status_((2,_201),_'B'),
 ((2, 201), 'C'): pilot_status_((2,_201),_'C'),
 ((2, 201), 'D'): pilot_status_((2,_201),_'D'),
 ((2, 201), 'E'): pilot_status_((2,_201),_'E'),
 ((3, 301), 'A'): pilot_status_((3,_301),_'A'),
 ((3, 301), 'B'): pilot_status_((3,_301),_'B'),
 ((3, 301), 'C'): pilot_status_((3,_301),_'C'),
 ((3, 301), 'D'): pilot_status_((3,_301),_'D'),
 ((3, 301), 'E'): pilot_status_((3,_301),_'E'),
 ((3, 302), 'A'): pilot_status_((3,_302)

We instantiate our model and use LpMinimize as the aim is to minimise costs.

In [199]:
model = pulp.LpProblem("PilotMinSchedProb", pulp.LpMinimize)

In our objective function we include our 2 costs: 
* Our variable costs is the product of the variable costs per unit and production
* Our fixed costs is the factory status - 1 (on) or 0 (off) - multiplied by the fixed cost of production

In [200]:
model += pulp.lpSum(
  1
)

We build up our constraints

In [217]:
sum(pilot_status[(line,x)]*pilot_quals.loc[x,'QUAL'] for x in pilots.index)

2*pilot_status_((9,_902),_'A') + 1*pilot_status_((9,_902),_'B') + 2*pilot_status_((9,_902),_'C') + 1*pilot_status_((9,_902),_'D') + 2*pilot_status_((9,_902),_'E') + 0

In [202]:
for line in lines.index:
    model += sum(pilot_status[(line,x)]*pilot_quals.loc[x,'QUAL'] for x in pilots.index) >= lines.loc[line, 'Requirement']

In [213]:
for line in lines.index:
    model += sum(pilot_status[(line,x)] for x in pilots.index) == 1 #only one pilot flying each line

In [226]:

pilot_status[((1,101),'A')]

pilot_status_((1,_101),_'A')

In [236]:
pilots

,1,2,3,4,5,6,7,8,9
PILOT,,,,,,,,,
A,0,1,0,1,1,0,1,0,0
B,0,1,0,1,1,0,1,1,1
C,1,0,1,1,1,0,1,1,1
D,0,0,1,1,0,1,0,0,0
E,1,0,1,0,1,0,1,0,0


In [214]:
model

PilotMinSchedProb:
MINIMIZE
0*__dummy + 1
SUBJECT TO
_C1: 2 pilot_status_((1,_101),_'A') + pilot_status_((1,_101),_'B')
 + 2 pilot_status_((1,_101),_'C') + pilot_status_((1,_101),_'D')
 + 2 pilot_status_((1,_101),_'E') >= 2

_C2: 2 pilot_status_((1,_102),_'A') + pilot_status_((1,_102),_'B')
 + 2 pilot_status_((1,_102),_'C') + pilot_status_((1,_102),_'D')
 + 2 pilot_status_((1,_102),_'E') >= 1

_C3: 2 pilot_status_((2,_201),_'A') + pilot_status_((2,_201),_'B')
 + 2 pilot_status_((2,_201),_'C') + pilot_status_((2,_201),_'D')
 + 2 pilot_status_((2,_201),_'E') >= 2

_C4: 2 pilot_status_((3,_301),_'A') + pilot_status_((3,_301),_'B')
 + 2 pilot_status_((3,_301),_'C') + pilot_status_((3,_301),_'D')
 + 2 pilot_status_((3,_301),_'E') >= 2

_C5: 2 pilot_status_((3,_302),_'A') + pilot_status_((3,_302),_'B')
 + 2 pilot_status_((3,_302),_'C') + pilot_status_((3,_302),_'D')
 + 2 pilot_status_((3,_302),_'E') >= 1

_C6: 2 pilot_status_((4,_401),_'A') + pilot_status_((4,_401),_'B')
 + 2 pilot_status_((

We then solve the model

In [215]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

Let's take a look at the optimal production schedule output for each month from each factory. For ease of viewing we'll output the data to a pandas DataFrame.

In [216]:
output = []
for Line, PILOT in pilot_status:
    if (pilot_status[(Line,PILOT)].varValue):
        var_output = {
            'Line': Line,
            'PILOT': PILOT,
        }
        output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Line'])
output_df.set_index(['Line', 'PILOT'], inplace=True)
output_df

,
Line,PILOT
"(1, 101)",E
"(1, 102)",E
"(2, 201)",A
"(3, 301)",A
"(3, 302)",E
"(4, 401)",C
"(5, 501)",A
"(5, 502)",A
"(6, 601)",E


In [ ]:
# Print our objective function value (Total Costs)
print (model.objective)